In [8]:
import pandas as pd

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from sklearn.model_selection import train_test_split

In [18]:
!pip3 install --upgrade transformers accelerate

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 9.3 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 419 kB 4.0 MB/s eta 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.21.3
    Uninstalling huggingface-hub-0.21.3:
      Successfully uninstalled huggingface-hub-0.21.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [23]:
import json
import requests

headers = {"Content-Type": "application/json"}

def text_generation(data, ip='localhost', port=None):
    resp = requests.put(f'http://{ip}:{port}/generate', data=json.dumps(data), headers=headers)
    return resp.json()


def get_generation(prompt, greedy, add_BOS, token_to_gen, min_tokens, temp, top_p, top_k, repetition, batch=False):
    data = {
        "sentences": [prompt] if not batch else prompt,
        "tokens_to_generate": int(token_to_gen),
        "temperature": temp,
        "add_BOS": add_BOS,
        "top_k": top_k,
        "top_p": top_p,
        "greedy": greedy,
        "all_probs": False,
        "repetition_penalty": repetition,
        "min_tokens_to_generate": int(min_tokens),
        "end_strings": ["<|endoftext|>", "<extra_id_1>", "\x11", "<extra_id_1>User"],
    }
    sentences = text_generation(data, port=1424)['sentences']
    return sentences[0] if not batch else sentences

PROMPT_TEMPLATE = """<extra_id_0>System

<extra_id_1>User
{prompt}
<extra_id_1>Assistant
"""

question = "Write a poem on NVIDIA in the style of Shakespeare"
prompt = PROMPT_TEMPLATE.format(prompt=question)
print(prompt)

response = get_generation(prompt, greedy=True, add_BOS=False, token_to_gen=1024, min_tokens=1, temp=1.0, top_p=1.0, top_k=0, repetition=1.0, batch=False)
response = response[len(prompt):]
if response.endswith("<extra_id_1>"):
    response = response[:-len("<extra_id_1>")]
print(response)


<extra_id_0>System

<extra_id_1>User
Write a poem on NVIDIA in the style of Shakespeare
<extra_id_1>Assistant



ConnectionError: HTTPConnectionPool(host='localhost', port=1424): Max retries exceeded with url: /generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x37cfbed30>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [20]:
import torch
print(torch.__version__)
print(torch.backends.mps.is_available())

2.3.1
True


In [21]:
device = torch.device("mps")
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.to(device)

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [1]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-27b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-27b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))


/Users/nikitaklenskij/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nikitaklenskij/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2-27b-it.
401 Client Error. (Request ID: Root=1-669e2d51-03631904508353995c1eb6d6;3d251fa4-78ea-4431-802d-c79f5e529835)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-27b-it/resolve/main/config.json.
Access to model google/gemma-2-27b-it is restricted. You must be authenticated to access it.

In [2]:
!pip3 install cohere

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 178 kB 178 kB/s eta 0:00:01
     |████████████████████████████████| 423 kB 250 kB/s eta 0:00:01
     |████████████████████████████████| 75 kB 295 kB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 507 kB/s eta 0:00:01
     |████████████████████████████████| 139 kB 320 kB/s eta 0:00:01
     |████████████████████████████████| 82 kB 844 kB/s eta 0:00:011
     |████████████████████████████████| 12.4 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 143 kB 581 kB/s eta 0:00:01
     |████████████████████████████████| 86 kB 969 kB/s eta 0:00:011
     |████████████████████████████████| 77 kB 979 kB/s eta 0:00:011
     |████████████████████████████████| 58 kB 752 kB/s eta 0:00:011
     |████████████████████████████████| 1.7 MB 682 kB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.

In [3]:
import cohere

co = cohere.Client(
    api_key="YOUR_API_KEY",
)

chat = co.chat(
    message="hello world!",
    model="command"
)

print(chat)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
!pip3 install huggingface-hub

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
!huggingface-cli download LiteLLMs/gemma-2b-GGUF Q4_0/Q4_0-00001-of-00009.gguf --local-dir . --local-dir-use-symlinks False


/Users/nikitaklenskij/Library/Python/3.9/lib/python/site-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Traceback (most recent call last):
  File "/Users/nikitaklenskij/Library/Python/3.9/lib/python/site-packages/huggingface_hub/utils/_errors.py", line 304, in hf_raise_for_status
    response.raise_for_status()
  File "/Users/nikitaklenskij/Library/Python/3.9/lib/python/site-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/LiteLLMs/gemma-2b-GGUF/resolve/main/Q4_0/Q4_0-00001-of-00009.gguf

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/nikitaklenskij/Library/Python/3.9/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())


In [6]:
!./main -ngl 35 -m Q4_0/Q4_0-00001-of-00009.gguf --color -c 8192 --temp 0.7 --repeat_penalty 1.1 -n -1 -p "<PROMPT>"


zsh:1: no such file or directory: ./main


In [8]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 3.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.83-cp310-cp310-macosx_14_0_arm64.whl size=2533990 sha256=ea677ce41d50a97f5baaf0531ef23be3ac5486da7f6d1fe86e41240cafc21e06
  Stored in directory: /Users/nikitaklenskij/Library/Caches/pip/wheels/25/97/95/bd309ea454a04b3b46c2b6321749172cf68a0279d892f12534
Successfully built llama-cpp-python


In [9]:
from llama_cpp import Llama
# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
  model_path="./Q4_0/Q4_0-00001-of-00009.gguf",  # Download the model file first
  n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
)
# Simple inference example
output = llm(
  "<PROMPT>", # Prompt
  max_tokens=512,  # Generate up to 512 tokens
  stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)
# Chat Completion API
llm = Llama(model_path="./Q4_0/Q4_0-00001-of-00009.gguf", chat_format="llama-2")  # Set chat_format according to the model you are using
llm.create_chat_completion(
    messages = [
        {"role": "system", "content": "You are a story writing assistant."},
        {
            "role": "user",
            "content": "Write a story about llamas."
        }
    ]
)


ModuleNotFoundError: No module named 'llama_cpp'

In [10]:
!huggingface-cli download bartowski/gemma-2-9b-it-GGUF --include "gemma-2-9b-it-Q4_K_M.gguf" --local-dir ./


Fetching 1 files:   0%|                                   | 0/1 [00:00<?, ?it/s]Downloading 'gemma-2-9b-it-Q4_K_M.gguf' to '.cache/huggingface/download/gemma-2-9b-it-Q4_K_M.gguf.13b2a7b4115bbd0900162edcebe476da1ba1fc24e718e8b40d32f6e300f56dfe.incomplete'

gemma-2-9b-it-Q4_K_M.gguf:   0%|                    | 0.00/5.76G [00:00<?, ?B/s]
gemma-2-9b-it-Q4_K_M.gguf:   0%|           | 10.5M/5.76G [00:00<06:11, 15.5MB/s]
gemma-2-9b-it-Q4_K_M.gguf:   0%|           | 21.0M/5.76G [00:01<07:59, 12.0MB/s]
gemma-2-9b-it-Q4_K_M.gguf:   1%|           | 31.5M/5.76G [00:02<07:01, 13.6MB/s]
gemma-2-9b-it-Q4_K_M.gguf:   1%|           | 41.9M/5.76G [00:03<07:20, 13.0MB/s]
gemma-2-9b-it-Q4_K_M.gguf:   1%|           | 52.4M/5.76G [00:04<07:36, 12.5MB/s]
gemma-2-9b-it-Q4_K_M.gguf:   1%|           | 62.9M/5.76G [00:04<07:38, 12.4MB/s]
gemma-2-9b-it-Q4_K_M.gguf:   1%|▏          | 73.4M/5.76G [00:05<07:39, 12.4MB/s]
gemma-2-9b-it-Q4_K_M.gguf:   1%|▏          | 83.9M/5.76G [00:06<07:42, 12.3MB/s]
gemma-2-9b-it-Q

In [ ]:
from llama_cpp import Llama
from rich import print

# Укажите путь к скачанной модели
model_path = "gemma-2-9b-it-Q4_K_M.gguf"

# Загрузка модели
model = Llama(model_path)



In [27]:
# Пример ввода
input_text = "Напиши мне диалог между заказчиком и исполнителем используя сарказм"

# Генерация ответа
output = model(input_text)

# Извлечение сгенерированного текста
generated = output['choices'][0]['text']

# Вывод ответа с использованием rich
print(generated_poem)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    5119.26 ms
llama_print_timings:      sample time =       1.83 ms /    16 runs   (    0.11 ms per token,  8733.62 tokens per second)
llama_print_timings: prompt eval time =    5406.75 ms /    14 tokens (  386.20 ms per token,     2.59 tokens per second)
llama_print_timings:        eval time =   90921.79 ms /    15 runs   ( 6061.45 ms per token,     0.16 tokens per second)
llama_print_timings:       total time =   96370.36 ms /    29 tokens


**Тема:**  Передача работы по веб-сайту

**За

In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
tokenizer = AutoTokenizer.from_pretrained(".cache/huggingface/download")

OSError: .cache/huggingface/download does not appear to have a file named config.json. Checkout 'https://huggingface.co/.cache/huggingface/download/tree/None' for available files.

In [1]:
from llama_cpp import Llama

# Укажите путь к скачанной модели
model_path = "gemma-2-9b-it-Q4_K_M.gguf"

# Загрузка модели с использованием CUDA
model = Llama(
    model_path,
    n_gpu_layers=10000000,  # Укажите максимально возможное значение
    n_batch=512,            # Увеличьте размер пакета
    n_threads=4             # Уменьшите количество потоков
)

# Функция для генерации ответа с увеличенной длиной ответа
def generate_response(context, max_tokens=50):
    output = model(context, max_tokens=max_tokens)
    response = output['choices'][0]['text']
    return response

# История сообщений
history = []

# Основной цикл чата
while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    history.append(f"User: {user_input}")
    context = "\n".join(history) + "\nBot:"
    response = generate_response(context, max_tokens=2)  # Увеличиваем длину ответа до 2048 токенов
    print(f"Bot: {response}")
    history.append(f"Bot: {response}")

llama_model_loader: loaded meta data with 33 key-value pairs and 464 tensors from gemma-2-9b-it-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma2
llama_model_loader: - kv   1:                               general.name str              = gemma-2-9b-it
llama_model_loader: - kv   2:                      gemma2.context_length u32              = 8192
llama_model_loader: - kv   3:                    gemma2.embedding_length u32              = 3584
llama_model_loader: - kv   4:                         gemma2.block_count u32              = 42
llama_model_loader: - kv   5:                 gemma2.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                gemma2.attention.head_count u32              = 16
llama_model_loader: - kv   7:             gemma2.attention.head_count_kv u32    

Bot:   Привет


In [31]:
import pandas as pd

In [2]:
df = pd.read_csv('data_stackoverflow.csv')

In [6]:
import pandas as pd
from llama_cpp import Llama

# Укажите путь к скачанной модели
model_path = "gemma-2-9b-it-Q4_K_M.gguf"

# Загрузка модели
model = Llama(model_path)

# Функция для классификации сообщения
def classify_message(message):
    context = f"Classify the message into two labels 0 - neutral, 1 - negative.:\n{message}\n Pay attention to sarcasm, disrespectful communication, first name (without patronymic), non-business communication, insult, ridicule, ridicule, messages with interjections em, ha, etc. Classification:"
    output = model(context, max_tokens=2)
    classification = output['choices'][0]['text'].strip()

    
    try:
        classification = int(classification)

    except:
        classification = - 1    

    return classification

# Загрузка данных
data = pd.read_csv('data_stackoverflow.csv')

# Классификация сообщений из колонки 'text'
for i in range (959, len(df)):
    new_label = classify_message(df.iloc[i]['text'])
    print(i, df.iloc[i]['text'], new_label)
    df.iat[i, df.columns.get_loc('label')] = new_label
# Сохранение результатов в новый CSV-файл
data.to_csv('data_stackoverflow_classified.csv', index=False)

# Вывод первых 5 строк с классификацией
print(data.head())

llama_model_loader: loaded meta data with 33 key-value pairs and 464 tensors from gemma-2-9b-it-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma2
llama_model_loader: - kv   1:                               general.name str              = gemma-2-9b-it
llama_model_loader: - kv   2:                      gemma2.context_length u32              = 8192
llama_model_loader: - kv   3:                    gemma2.embedding_length u32              = 3584
llama_model_loader: - kv   4:                         gemma2.block_count u32              = 42
llama_model_loader: - kv   5:                 gemma2.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                gemma2.attention.head_count u32              = 16
llama_model_loader: - kv   7:             gemma2.attention.head_count_kv u32    

959 а что ввели для вычислений? - а ничего -1



llama_print_timings:        load time =    4608.94 ms
llama_print_timings:      sample time =       0.18 ms /     2 runs   (    0.09 ms per token, 11111.11 tokens per second)
llama_print_timings: prompt eval time =    2997.50 ms /    81 tokens (   37.01 ms per token,    27.02 tokens per second)
llama_print_timings:        eval time =      68.53 ms /     1 runs   (   68.53 ms per token,    14.59 tokens per second)
llama_print_timings:       total time =    3067.55 ms /    82 tokens
Llama.generate: prefix-match hit


960 эм, как вы представляете себе принудительный обрыв выполнения кода? 0



llama_print_timings:        load time =    4608.94 ms
llama_print_timings:      sample time =       0.18 ms /     2 runs   (    0.09 ms per token, 11428.57 tokens per second)
llama_print_timings: prompt eval time =    3452.11 ms /   119 tokens (   29.01 ms per token,    34.47 tokens per second)
llama_print_timings:        eval time =     135.07 ms /     1 runs   (  135.07 ms per token,     7.40 tokens per second)
llama_print_timings:       total time =    3589.61 ms /   120 tokens
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    4608.94 ms
llama_print_timings:      sample time =       0.18 ms /     2 runs   (    0.09 ms per token, 11299.44 tokens per second)
llama_print_timings: prompt eval time =    4439.58 ms /   166 tokens (   26.74 ms per token,    37.39 tokens per second)
llama_print_timings:        eval time =      77.75 ms /     1 runs   (   77.75 ms per token,    12.86 tokens per second)
llama_print_timings:       total time =    4519.12 ms /   167 tokens
Llama.generate: prefix-match hit


964 Еще лучше разобраться с идеей расщепления логики посредством функций вам поможет понимание работы функций в математике - вы можете не знать, как устроен синус, например, но "швырнув" в него аргумент вы получаете какой-то результат; или вы можете использовать некоторое имя "f()" для обозначения какой-нибудь очень сложной мат.функции, не дублируя каждый раз ее целиком. Примерно так же и в программировании. -1



llama_print_timings:        load time =    4608.94 ms
llama_print_timings:      sample time =       0.19 ms /     2 runs   (    0.09 ms per token, 10582.01 tokens per second)
llama_print_timings: prompt eval time =    4752.83 ms /   169 tokens (   28.12 ms per token,    35.56 tokens per second)
llama_print_timings:        eval time =     272.32 ms /     1 runs   (  272.32 ms per token,     3.67 tokens per second)
llama_print_timings:       total time =    5028.49 ms /   170 tokens
Llama.generate: prefix-match hit


965 +  DRY - don't repeat yourself - "не повторяй себя" или "не повторяйся". Гораздо удобнее написать имя функции нежели комбинацию из 10 и более команд в 30 различных точках основной программы. А если таких точек ещё больше, и параметры в них должны быть отличные друг от друга - то тут вообще без функций можно сойти с ума. Или упасть без сил после часов беспрерывного набора одного и того же кода с вариациями. -1


KeyboardInterrupt: 

In [84]:
df_short = df[:1131]

In [87]:
print(df_short)

text  label
0     Ну, в стандарте такого нет, а вот что в этом <...     -1
1     @Harry этот оператор из С++20, насколько я помню.     -1
2     @Harry пишет "превышение допустимой длины на 2...      2
3     @RiotBr3aker А! ну, я еще и С++17 не очень выу...     -1
4     Если у кого появится желание перевести: stacko...      0
...                                                 ...    ...
1126  static_cast делает только преобразование арифм...      0
1127  @IAZ: частично: ideone.com/wkpEhT Спасибо, поп...      1
1128  @VladD: static_cast для указателей работает "п...      2
1129  @avp: "не знаешь - не трогай" -- очень правиль...     -1
1130  Что-то я подозреваю, что пустая структура при ...      1

[1131 rows x 2 columns]

In [64]:
print(df)

text  label
0      Ну, в стандарте такого нет, а вот что в этом <...      3
1      @Harry этот оператор из С++20, насколько я помню.     -1
2      @Harry пишет "превышение допустимой длины на 2...      2
3      @RiotBr3aker А! ну, я еще и С++17 не очень выу...     -1
4      Если у кого появится желание перевести: stacko...      0
...                                                  ...    ...
46347                                  Тоже правильно :)     -1
46348  1. Смысл использовать jquery только для провер...     -1
46349  1. алерт только для того, чтобы программист са...     -1
46350  @Air вы действительно считаете, что этот пост ...      2
46351  @PashaPash, не так не считаю, это просто невни...     -1

[46352 rows x 2 columns]

In [94]:
df_short = pd.read_csv('data_stackoverflow_short_gemma.csv')

In [91]:
df_short = df_short[df_short['label'] != -1]

In [95]:
print(df_short)

text  label
0    @Harry пишет "превышение допустимой длины на 2...      2
1    Если у кого появится желание перевести: stacko...      0
2    Эм.. Это ж неправда... Даже про целочисленное ...      2
3    Машинный перевод каждый может применить к ориг...      2
4    Ничего не понял из ответа. Линк на оригинал по...      2
..                                                 ...    ...
830  Хороший ответ.  --  А ведь можно сказать и так...      2
831  static_cast делает только преобразование арифм...      0
832  @IAZ: частично: ideone.com/wkpEhT Спасибо, поп...      1
833  @VladD: static_cast для указателей работает "п...      2
834  Что-то я подозреваю, что пустая структура при ...      1

[835 rows x 2 columns]